## NTUEE儀表板資料處理中心

##### 作者：NTUEE儀表板資料組

#### 引入必要套件與函數

In [62]:
!pip3 install pandas
!pip3 install geocoder==1.38.1
!pip3 install geopandas==0.13.2
!pip3 install shapely==2.0.2



In [63]:
# import packages needed
import requests
import pandas as pd
import geocoder
import geopandas as gpd
from shapely.geometry import Point, Polygon

##### 獲取臺北大數據中心資料：```get_datataipei_api```


In [5]:


def get_datataipei_api(rid):
    '''
    Get Data.taipei API，自動遍歷所有資料。
    (data.taipei的API單次return最多1000筆，因此需利用offset定位，取得所有資料)
    '''
    url = f"""https://data.taipei/api/v1/dataset/{rid}?scope=resourceAquire"""
    response = requests.request("GET", url)
    data_dict = response.json()
    count = data_dict['result']['count']
    res = list()
    offset_count = int(count/1000)
    for i in range(offset_count+1):
        i = i* 1000
        url = f"""https://data.taipei/api/v1/dataset/{rid}?scope=resourceAquire&offset={i}&limit=1000"""
        response = requests.request("GET", url)
        get_json = response.json()
        res.extend(get_json['result']['results'])
    return pd.DataFrame(res)

##### 資料庫（不斷更新！）


In [6]:
dataset_api = {
    "年齡 & 性別分布": {
        "粗略(幼青老年)": "https://data.taipei/dataset/detail?id=1e0c58e9-6aa5-4acb-a5a1-f60bacad60f3",
        "詳細(年齡)": "https://data.taipei/dataset/detail?id=64c8a3a0-3b9a-4f49-a13a-fb1eb2ffa4b1",
        "詳細": "https://data.taipei/dataset/detail?id=a6394e3f-3514-4542-87bd-de4310a40db3",
    },
    "疾病": {
        "年分-疾病類-人數": "https://data.taipei/dataset/detail?id=585e070a-50bf-419d-b521-a103446f4496",
        "較舊年份": "https://data.taipei/dataset/detail?id=eeb730bc-4764-46e5-a2a2-9bdf6dc3912a",
    },
    "醫療機構": {
        "按科別的醫療院所分布": "搜尋相關科別",
        "健康服務中心": "https://data.taipei/dataset/detail?id=d4c6d4e0-c2b4-48e3-99b4-f670d820326b",
        "老人特約醫院": {
            "臺北市老人健康檢查特約醫事機構": "https://data.taipei/dataset/detail?id=9a1aefde-2e4a-467c-9ec4-256e33ad6df0",
            "臺北市銀髮族服務_老人收容安置補助名冊": "https://data.taipei/dataset/detail?id=23aff01f-67a3-461f-9fda-2fafe4ec9284",
        },
        "慢性病": {
            "臺北市腎臟病健康促進機構名單": "https://data.taipei/dataset/detail?id=5cf845a6-e9d0-42ed-861f-8c88bb1b694c",
            "臺北市糖尿病共照網醫事機構名單": "https://data.taipei/dataset/detail?id=e6f6ced6-4d66-4be4-93af-57b71b17b2b0",
        },
        "醫療糾紛": "https://data.taipei/dataset/detail?id=289439b3-35c8-42b3-a4c7-db39df17dcd2",
    },
    "補助 & 救助": {
        "中低收入戶": {
            "戶數、金額": "https://data.taipei/dataset/detail?id=0b29acd6-9609-48be-984b-dd1281d1f04e",
            "項目、人數": "https://data.taipei/dataset/detail?id=485e17e5-472f-4526-ab5f-b4a9d58b43d8",
            "臺北市低收入戶家庭補助(87下半年以後)": "https://data.taipei/dataset/detail?id=27972a0d-707f-4e70-ba64-28c6fce5a23b",
        },
        "醫療補助": {
            "臺北市市民醫療補助": "https://data.taipei/dataset/detail?id=c0eed3c6-79f2-499a-8269-0c0d4348ccb3",
            "臺北市社會救助醫療及看護補助": "https://data.taipei/dataset/detail?id=dba8d919-b730-4723-a6a5-c1a451275873",
            "社會福利統計_臺北市市民醫療補助": "https://data.taipei/dataset/detail?id=c5e6bff4-5907-4e60-91ed-ea88bc8dc063",
            "臺北市市民醫療及急難救助按月別": "https://data.taipei/dataset/detail?id=652dcd39-3375-469d-bbc4-63573932a718",
            "臺北市市民急難救助": "https://data.taipei/dataset/detail?id=f5db7596-8fb6-40cf-8b7d-1925ddb513c9",
        },
    },
    "長照": {
        "長照喘息服務特約單位": "https://data.taipei/dataset/detail?id=959c34b2-9698-4d99-8987-d389efe18c04",
        "長照專業服務特約單位": "https://data.taipei/dataset/detail?id=d3ba4f78-fb09-47ec-867c-53110570d12f",
        "臺北市立案之一般護理之家一覽表": "https://health.gov.taipei/News_Content.aspx?n=B283D71AA0A7D98A&sms=EDD21D8B4B037BC3&s=3B9E85AB87282057",
        "臺北市長者長期照顧、安養機構及老人公寓概況按月別": "https://data.taipei/dataset/detail?id=af257faf-9ee6-4ca7-9631-4e4cb7d4e2fd",
        "長照機構": {
            "臺北市銀髮族服務_私立老人安養暨長期照顧機構": "9a3e0440-87ea-4793-9240-5c8dc02b5129",
            "臺北市銀髮族服務_公立老人安養暨長期照顧機構": "67549b46-4328-48d3-973f-dd7d3a96dbed",
            "臺北市銀髮族服務_公設民營老人安養暨長期照顧機構": "2aa363e1-6bc3-4312-8388-c4f5e25c7149",
        },
    },
    "老人": {
        "臺北市獨居老人(89－105年)": "https://data.taipei/dataset/detail?id=2b4d80c1-29d8-4647-ad2d-9c2658478462",
        "臺北市獨居老人(106年以後)": "https://data.taipei/dataset/detail?id=d4833dcf-0cd1-455e-99b9-ba3bbf23fbe9",
    },
}



### 1. 步步為銀

##### 點子
1. 長照機構散佈建立的方便程度位能場
2. （獨居）老人散佈圖

##### 長照機構散佈建立的方便程度位能場

流程：

1. 利用`get_datataipei_api`取得安養機構資料
2. 取得地址資料，利用`geocoder`轉成經緯度，再轉成WGS84/TWD97座標
3. 取得床位資料，以該安養機構為中心，利用`shapely`創建不透明度平方反比遞減的圓，處理重疊的透明度疊加
4. 將這些圓物件存成`.geojson`

In [75]:

# 取得資料
私立老人安養機構資料 = get_datataipei_api(dataset_api["長照"]["長照機構"]["臺北市銀髮族服務_私立老人安養暨長期照顧機構"])
公立老人安養機構資料 = get_datataipei_api(dataset_api["長照"]["長照機構"]["臺北市銀髮族服務_公立老人安養暨長期照顧機構"])
公設民營老人安養機構資料 = get_datataipei_api(dataset_api["長照"]["長照機構"]["臺北市銀髮族服務_公設民營老人安養暨長期照顧機構"])
老人安養機構資料 = pd.concat([私立老人安養機構資料, 公立老人安養機構資料, 公設民營老人安養機構資料])
老人安養機構資料.head(23)

,_id,_importdate,屬性,機構名稱,區域別,地址,電話,收容對象,核定總床位數量,長照床位數量,養護床位數量,失智床位數量,安養床位數量
0,1,"{'date': '2023-07-06 19:00:56.960295', 'timezo...",私立,臺北市私立上美老人長期照顧中心(養護型),士林區,臺北市士林區重慶北路4段162號1-4樓,(02)28122366,養護,33,0,33,0,0
1,2,"{'date': '2023-07-06 19:00:56.971035', 'timezo...",私立,臺北市私立璞園老人長期照顧中心(養護型),士林區,臺北市士林區富光里010鄰葫蘆街33號4、5、6樓,(02)28163696,養護,46,0,46,0,0
2,3,"{'date': '2023-07-06 19:00:56.972223', 'timezo...",私立,臺北市私立祐心老人長期照顧中心(養護型),士林區,臺北市士林區名山里 009鄰至誠路1段305巷3弄14號1-2樓,(02)28310432,養護,16,0,16,0,0
3,4,"{'date': '2023-07-06 19:00:56.973572', 'timezo...",私立,臺北市私立永青老人長期照顧中心(養護型),士林區,臺北市士林區天壽里003鄰中山北路6段427巷8號1-4樓,(02)28761908,養護、長照,31,13,18,0,0
4,5,"{'date': '2023-07-06 19:00:56.974947', 'timezo...",私立,臺北市私立祇福老人養護所,士林區,臺北市士林區三玉里016鄰忠誠路2段10巷11、13號1-2樓,(02)28317713,養護,39,0,39,0,0
5,6,"{'date': '2023-07-06 19:00:56.976219', 'timezo...",私立,臺北市私立荷園老人長期照顧中心(養護型),士林區,臺北市士林區富光里010鄰葫蘆街33號2-3樓,(02)28163696,養護,39,0,39,0,0
6,7,"{'date': '2023-07-06 19:00:56.977645', 'timezo...",私立,臺北市私立柏安老人養護所,士林區,臺北市士林區蘭雅里 024鄰 中山北路6段290巷52、54號1樓,(02)28333536,養護,20,0,20,0,0
7,8,"{'date': '2023-07-06 19:00:56.979220', 'timezo...",私立,臺北市私立松園老人長期照顧中心(養護型),士林區,臺北市士林區蘭雅里 020鄰德行東路61巷1、2、3號1-2樓,(02)88662097,養護,49,0,49,0,0
8,9,"{'date': '2023-07-06 19:00:56.980526', 'timezo...",私立,臺北市私立天玉老人長期照顧中心(長期照護型),士林區,臺北市士林區天玉街38巷18弄18號1-6樓,(02)28720239,長照,39,39,0,0,0
9,10,"{'date': '2023-07-06 19:00:56.981839', 'timezo...",私立,臺北市私立慧光老人養護所,大同區,臺北市大同區星明里 014鄰寧夏路32 號5樓,(02)25582275,養護,40,0,40,0,0


In [107]:

# 要跑10分鐘...
老人安養機構資料['經緯度'] = 老人安養機構資料['地址'].apply(
    lambda x: Point(
        geocoder.arcgis(x).json['lng'] ,
        geocoder.arcgis(x).json['lat'] 
    ) if geocoder.arcgis(x) and geocoder.arcgis(x).json else None
)

KeyboardInterrupt: 

In [106]:
老人安養機構資料

,_id,_importdate,屬性,機構名稱,區域別,地址,電話,收容對象,核定總床位數量,長照床位數量,養護床位數量,失智床位數量,安養床位數量
0,1,"{'date': '2023-07-06 19:00:56.960295', 'timezo...",私立,臺北市私立上美老人長期照顧中心(養護型),士林區,臺北市士林區重慶北路4段162號1-4樓,(02)28122366,養護,33,0,33,0,0
1,2,"{'date': '2023-07-06 19:00:56.971035', 'timezo...",私立,臺北市私立璞園老人長期照顧中心(養護型),士林區,臺北市士林區富光里010鄰葫蘆街33號4、5、6樓,(02)28163696,養護,46,0,46,0,0
2,3,"{'date': '2023-07-06 19:00:56.972223', 'timezo...",私立,臺北市私立祐心老人長期照顧中心(養護型),士林區,臺北市士林區名山里 009鄰至誠路1段305巷3弄14號1-2樓,(02)28310432,養護,16,0,16,0,0
3,4,"{'date': '2023-07-06 19:00:56.973572', 'timezo...",私立,臺北市私立永青老人長期照顧中心(養護型),士林區,臺北市士林區天壽里003鄰中山北路6段427巷8號1-4樓,(02)28761908,養護、長照,31,13,18,0,0
4,5,"{'date': '2023-07-06 19:00:56.974947', 'timezo...",私立,臺北市私立祇福老人養護所,士林區,臺北市士林區三玉里016鄰忠誠路2段10巷11、13號1-2樓,(02)28317713,養護,39,0,39,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1,"{'date': '2023-07-06 19:02:44.100268', 'timezo...",公立,臺北市政府社會局老人自費安養中心,文山區,臺北市文山區忠順里009鄰興隆路4段109巷30弄6號,(02)29393146,安養,366,0,0,0,366
1,2,"{'date': '2023-07-06 19:02:44.111820', 'timezo...",公立,臺北市立浩然敬老院,北投區,臺北市北投區關渡里016鄰知行路75號,(02)28581081,安養\n養護,400,0,175,0,225
0,1,"{'date': '2023-07-06 19:03:57.776525', 'timezo...",公設民營,臺北市至善老人安養護中心,士林區,臺北市士林區永福里001鄰仰德大道2段2巷50號,(02)28832666,安養\n養護\n長照,471,124,194,0,153
1,2,"{'date': '2023-07-06 19:03:57.785953', 'timezo...",公設民營,臺北市文山老人養護中心,文山區,臺北市文山區興豐里016鄰興隆路2段95巷8號3~6樓,(02)89313190,養護 \n,46,0,46,0,0


### 2. 臺北歡迎您

### 3. 宜居臺北

### 4. 祝您好孕

### 5. 韌性城市